# Implement a Convolutional Neural Network (CNN)

### Import needed modules

In [15]:
import torch

# Inner network modules, loss functions, etc.
import torch.nn as nn
# Optimization algorithms
import torch.optim as optim
# Dataset manager
from torch.utils.data import DataLoader
# Datasets
import torchvision.datasets as datasets
# Dataset transformations
import torchvision.transforms as transforms

import torch.cuda as cuda

### Check device

In [3]:
device = "cuda" if cuda.is_available() else "cpu"
if device == "cuda":
  print(f"{device} - {cuda.get_device_name()}")
else:
  print(f"{device}")

cuda - NVIDIA GeForce MX130


## 0. Define and prepare Data

In [4]:
train_dataset = datasets.MNIST(root='dataset/', download=False,train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='dataset/', download=False,train=False, transform=transforms.ToTensor())
print(train_dataset)
print(test_dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()


### Set input and output sizes

In [5]:
input_size = 28 * 28 # MNIST dataset
num_classes = 10 # In this classification problem we have 10 classes or categories as output

## 1. Create Model

### Create model of a basic Neural Network with 1 hidden layer.

In [6]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        # Input to hidden layer
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU()
        )
        # Hidden to output layer
        self.layer2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

### Create model a Convolutional Neural Network

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(3,3), stride=(1,1), padding=(1,1))

### Initialize a NN 

In [7]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)
model

NN(
  (layer1): Sequential(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
  )
  (layer2): Linear(in_features=50, out_features=10, bias=True)
)

## 2. Loss and optimizers

### Hyperparameters

In [23]:
lr = 0.001 # Learning rate
batch_size = 64
num_epochs = 10

### Loss and optimizers

In [9]:
criterion = nn.CrossEntropyLoss() # For classification problems
optimizer = optim.Adam(model.parameters(), lr=lr)

## 3. Training

### Load data

In [10]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

### Training loop

In [24]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Send data and targets to device
        data = data.to(device)
        targets = targets.to(device)

        # Flatten the matrix (image) to a single dimension
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets) # Compare the output of the model with the real ones (targets)

        # Backward
        optimizer.zero_grad() # Reset gradients to zero
        loss.backward() # Backpropagation of loss
        optimizer.step() # Update weights

        # On each epoch at the first mini-batch
        if batch_idx == 0:
            # Print epoch and loss
            print(
                f'Epoch: {epoch} / {num_epochs} '
                f'Loss: {loss}'
            )

Epoch: 0 / 10Loss: 0.267354816198349
Epoch: 1 / 10Loss: 0.11021119356155396
Epoch: 2 / 10Loss: 0.06108015403151512
Epoch: 3 / 10Loss: 0.05317104235291481
Epoch: 4 / 10Loss: 0.09522654861211777
Epoch: 5 / 10Loss: 0.028607942163944244
Epoch: 6 / 10Loss: 0.07075036317110062
Epoch: 7 / 10Loss: 0.03927686810493469
Epoch: 8 / 10Loss: 0.021634917706251144
Epoch: 9 / 10Loss: 0.018378451466560364


## 4. Validation

### Load test data

In [33]:
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Check accuracy

In [34]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on testing data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y, in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()
    return

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59341 / 60000 with accuracy 98.90
Checking accuracy on testing data
Got 9722 / 10000 with accuracy 97.22
